# Data

In this jupyter notebook the necessary code to run the experiments will be found. 

## Imports

The code snippet provided sets up the environment by installing matplotlib for data visualization and scapy for packet crafting and network traffic analysis. It imports various Python libraries such as ast for processing tree-based data structures, ipaddress for handling IPv4 addresses and networks, and json for manipulating JSON data formats. The script also utilizes matplotlib for plotting and pandas for data handling to create visual representations of network data. Notably, it includes custom modules like EPHeader and MRI, presumably for enhanced packet header manipulation and Magnetic Resonance Imaging data processing within network packets, respectively. The code also initializes the fablib_manager from the fabrictestbed_extensions library, which likely facilitates interaction with a fabric testbed environment, enabling hands-on experiments with virtual network configurations and behavior.

In [ ]:
!pip install matplotlib
!pip3 install scapy

import ast
import ipaddress
import json
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import pandas as pd
import re
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from io import StringIO
from ipaddress import IPv4Address, IPv4Network
from scapy.all import *
from time import sleep
from utils.EPHeader import *
from utils.MRI import *

## Variables, nodes and interfaces

This Python code manages network configurations within a specified network slice named "INT" using the fablib library. It reads from a JSON file to retrieve MAC and IP addresses for various hosts and switches across different subnets. The script defines network parameters, including expanding subnet masks for broader network coverage. It identifies network devices and their interfaces, facilitating detailed management of connections between hosts and switches. 

In [ ]:
verbose = 0
slice_name = "INT"  # Slice name

fablib = fablib_manager()
slice = fablib.get_slice(name=slice_name)

file_path = './utils/network_data.json'

with open(file_path, 'r') as file:
    data = json.load(file)

    # MAC
    h1_mac = data['hosts']['h1']['mac']
    h2_mac = data['hosts']['h2']['mac']
    h3_mac = data['hosts']['h3']['mac']
    h4_mac = data['hosts']['h4']['mac']

    s1_mac_1 = data['switches']['s1']['macs'][0]
    s1_mac_2 = data['switches']['s1']['macs'][1]
    s1_mac_3 = data['switches']['s1']['macs'][2]
    s1_mac_4 = data['switches']['s1']['macs'][3]

    s2_mac_1 = data['switches']['s2']['macs'][0]
    s2_mac_2 = data['switches']['s2']['macs'][1]
    s2_mac_3 = data['switches']['s2']['macs'][2]
    s2_mac_4 = data['switches']['s2']['macs'][3]

    s3_mac_1 = data['switches']['s3']['macs'][0]
    s3_mac_2 = data['switches']['s3']['macs'][1]
    s3_mac_3 = data['switches']['s3']['macs'][2]
    s3_mac_4 = data['switches']['s3']['macs'][3]

    s4_mac_1 = data['switches']['s4']['macs'][0]
    s4_mac_2 = data['switches']['s4']['macs'][1]
    s4_mac_3 = data['switches']['s4']['macs'][2]
    s4_mac_4 = data['switches']['s4']['macs'][3]

    # IP
    subnet1 = data['subnets']['subnet1']
    h1_ip = data['hosts']['h1']['ip']
    s1_ip = data['switches']['s1']['ip']

    subnet2 = data['subnets']['subnet2']
    h2_ip = data['hosts']['h2']['ip']
    s2_ip = data['switches']['s2']['ip']
    
    subnet3 = data['subnets']['subnet3']
    h3_ip = data['hosts']['h3']['ip']
    s3_ip = data['switches']['s3']['ip']
    
    subnet4 = data['subnets']['subnet4']
    h4_ip = data['hosts']['h4']['ip']
    s4_ip = data['switches']['s4']['ip']

network = ipaddress.IPv4Network(subnet1)
net = network.supernet(new_prefix=16)

s1 = slice.get_node(name='s1')
s2 = slice.get_node(name='s2')
s3 = slice.get_node(name='s3')
s4 = slice.get_node(name='s4')

h1 = slice.get_node(name='h1')
h2 = slice.get_node(name='h2')
h3 = slice.get_node(name='h3')
h4 = slice.get_node(name='h4')

s1_iface1_name = s1.get_interface(network_name='s1h1').get_device_name()
s1_iface2_name = s1.get_interface(network_name='s1s2').get_device_name()
s1_iface3_name = s1.get_interface(network_name='s1s3').get_device_name()
s1_iface4_name = s1.get_interface(network_name='s1s4').get_device_name()
s2_iface1_name = s2.get_interface(network_name='s1s2').get_device_name()
s2_iface2_name = s2.get_interface(network_name='s2h2').get_device_name()
s2_iface3_name = s2.get_interface(network_name='s2s3').get_device_name()
s2_iface4_name = s2.get_interface(network_name='s2s4').get_device_name()
s3_iface1_name = s3.get_interface(network_name='s1s3').get_device_name()
s3_iface2_name = s3.get_interface(network_name='s2s3').get_device_name()
s3_iface3_name = s3.get_interface(network_name='s3h3').get_device_name()
s3_iface4_name = s3.get_interface(network_name='s3s4').get_device_name()
s4_iface1_name = s4.get_interface(network_name='s1s4').get_device_name()
s4_iface2_name = s4.get_interface(network_name='s2s4').get_device_name()
s4_iface3_name = s4.get_interface(network_name='s3s4').get_device_name()
s4_iface4_name = s4.get_interface(network_name='s4h4').get_device_name()
h1_iface0_name = h1.get_interface(network_name='s1h1').get_device_name()
h2_iface0_name = h2.get_interface(network_name='s2h2').get_device_name()
h3_iface0_name = h3.get_interface(network_name='s3h3').get_device_name()
h4_iface0_name = h4.get_interface(network_name='s4h4').get_device_name()

In [ ]:
stdout, stderr = s1.execute("sudo apt install -y python3-pip", quiet=True)
print("s1")
stdout, stderr = s2.execute("sudo apt install -y python3-pip", quiet=True)
print("s2")
stdout, stderr = h1.execute("sudo apt install -y python3-pip", quiet=True)
print("h1")
stdout, stderr = h2.execute("sudo apt install -y python3-pip", quiet=True)
print("h2")

In [ ]:
stdout, stderr = s1.execute("sudo pip3 install pandas", quiet=True)
print("s1")
stdout, stderr = s2.execute("sudo pip3 install pandas", quiet=True)
print("s2")
stdout, stderr = h1.execute("sudo pip3 install pandas", quiet=True)
print("h1")
stdout, stderr = h2.execute("sudo pip3 install pandas", quiet=True)
print("h2")

In [ ]:
stdout, stderr = h1.execute("sudo apt install -y iperf3", quiet=True)
print("h1")
stdout, stderr = h2.execute("sudo apt install -y iperf3", quiet=True)
print("h2")

In [ ]:
stdout, stderr = h1.execute("sudo apt-get update", quiet=True)
stdout, stderr = h1.execute("sudo apt-get install ntp", quiet=True)
stdout, stderr = h1.execute("sudo systemctl restart ntp", quiet=True)

stdout, stderr = h2.execute("sudo apt-get update", quiet=True)
stdout, stderr = h2.execute("sudo apt-get install ntp", quiet=True)
stdout, stderr = h2.execute("sudo systemctl restart ntp", quiet=True)

stdout, stderr = s1.execute("sudo apt-get update", quiet=True)
stdout, stderr = s1.execute("sudo apt-get install ntp", quiet=True)
stdout, stderr = s1.execute("sudo systemctl restart ntp", quiet=True)

stdout, stderr = s2.execute("sudo apt-get update", quiet=True)
stdout, stderr = s2.execute("sudo apt-get install ntp", quiet=True)
stdout, stderr = s2.execute("sudo systemctl restart ntp", quiet=True)

## Experiment

The code used in each of the experiments is outlined below. Initially, the necessary files will be transferred to the nodes. Once the files are in place, the experiment can commence. The experiment involves three main steps:

- Data Collection: Execute specific files to gather initial information from the nodes.
- Traffic Generation: Launch the corresponding network traffic based on the experiment's requirements.
- Data Processing and Storage: Process the extracted data from the configuration files and save the results in CSV format.
  
By following these steps, we ensure a structured approach to conducting the experiments, capturing relevant data, and organizing the results for analysis.

### Upload packets

This Python code imports the Uploader class from a module named utils.uploader and creates an instance of the Uploader class, assigning it to the variable up. This instance can now be used to upload the file (second argument) to the node (first argument).

In [ ]:
from utils.uploader import Uploader
up = Uploader()

In the following code we are going to upload to each one of the nodes a python script called "nPackets.py". This Python script utilizes the Scapy library to analyze packet capture files (.pcap) for network diagnostics. It reads a specified pcap file and extracts packets based on specific protocols—UDP and a custom protocol indicated by TYPE_EPHeader. For each protocol type, the script computes statistics such as the total number of packets, minimum, average, and maximum packet sizes. These statistics are generated for both UDP packets and packets matching the custom protocol, providing insight into the traffic characteristics of each. The script is designed for command-line execution with the pcap file path as an argument, making it suitable for automated network analysis tasks. 

In [ ]:
up.upload("h1", "./graphs/nPackets.py")
up.upload("h2", "./graphs/nPackets.py")
up.upload("s1", "./graphs/nPackets.py")
up.upload("s2", "./graphs/nPackets.py")

In the following code we are going to upload to "h1" a python script called "BgT_iperf.py".The Python script serves as a command-line tool to facilitate network performance testing using the iperf3 utility in client mode with UDP packets. It accepts three parameters: the destination server address, the test duration in seconds, and the number of parallel client threads, all of which are managed through argparse for command-line integration. The script constructs an iperf3 command with these parameters and executes it, capturing the output in JSON format for detailed performance analysis. This output is then printed to the console, and the script ensures the buffer is flushed immediately for real-time result viewing.

In [ ]:
up.upload("h1", "./graphs/BgT_iperf.py")

In the following code we are going to upload to "h2" a python script called "receive.py".The Python script uses the Scapy library to monitor and analyze network packets tailored to a custom protocol. It selects a network interface, specifically looking for "enp7s0", and defines custom packet structures, including `EPHeader` and `SwitchTrace`, to handle specialized network data. The script captures packets on the specified interface that contain a unique protocol identifier, converting captured packets to hexadecimal for analysis and logging them to a pcap file.

In [ ]:
up.upload("h2", "./graphs/receive.py")

In the following code we are going to upload to "h1" a python script called "sendPackets.py". This Python script leverages the Scapy library to create and transmit custom network packets for testing purposes. The script constructs packets comprising Ethernet and IP layers, including a specialized EPHeader and an IPOption_MRI for multipath routing information. Each packet is populated with predefined values and sent in a loop based on user input, with a one-second delay between transmissions.

In [ ]:
up.upload("h1", "./graphs/sendPacket.py")

### Start experiment

The first line executes a command to terminate all iperf3 processes silently in h2, while the second line does the same for all Python 3 instances, capturing any output or errors into stdout and stderr respectively. 

In [ ]:
h2.execute("sudo killall iperf3", quiet=True)
stdout, stderr = h2.execute("sudo killall python3", quiet=True)

The Python code snippet launches an iperf3 server instance on "h2" in a separate thread. This is accomplished using a method called execute_thread, which likely allows the iperf3 -s command (indicating server mode) to run asynchronously, ensuring that the main program can continue executing other tasks without waiting for the iperf3 server process to finish.

In [ ]:
iperf2 = h2.execute_thread(f"iperf3 -s")

In the code snippet, three variables are set up for a configuration or script that appears to be preparing for a network performance testing scenario:

- experimentNumber: This variable likely indicates a identifier for the experiment.

- tiperf: This variable specifies the duration in seconds for the iperf3 command. 

- nthreads: This variable defines the number of parallel client threads to be used in the test. 

These variables are essential for controlling the parameters of a network testing session, allowing for precise adjustments to the testing environment based on the needs of the experiment or the specifics of the network being tested.

In [ ]:
experimentNumber = 28
tiperf = 10
nthreads = 4

The Python code snippet launches a script named receive.py on "h2" using elevated privileges (sudo). The script is run asynchronously in a separate thread by the method execute_thread, allowing the main program to continue its execution without waiting for receive.py to complete.

In [ ]:
mriPacket_receive = h2.execute_thread(f"sudo python3 receive.py")

The following cell orchestrates parallel network traffic captures across multiple devices—two hosts and two switches—using `tcpdump`, a command-line packet analyzer. Commands are executed asynchronously in separate threads to prevent blocking, enabling simultaneous captures on specified network interfaces for each device. For hosts `h1` and `h2`, and switches `s1` and `s2`, traffic is recorded on their respective interfaces, with each session outputting to a dedicated pcap file.

In [ ]:
h1.execute_thread(f"sudo tcpdump -i {h1_iface0_name} -w h1_0.pcap")
h2.execute_thread(f"sudo tcpdump -i {h2_iface0_name} -w h2_0.pcap")
print("hosts")
s1.execute_thread(f"sudo tcpdump -i {s1_iface1_name} -w s1_1.pcap")
s1.execute_thread(f"sudo tcpdump -i {s1_iface2_name} -w s1_2.pcap")
print("s1")
s2.execute_thread(f"sudo tcpdump -i {s2_iface1_name} -w s2_1.pcap")
s2.execute_thread(f"sudo tcpdump -i {s2_iface2_name} -w s2_2.pcap")
print("s2")

The code snippet initiates an asynchronous task on the host "h1" to execute a Python script named "sendPacket.py" using elevated privileges (`sudo`). This script send packets by passing three arguments: `1`, `2`, and `30`. The first one is represents the host that is sending the packet. The second argument, in this case "2", is the host destination. And the last argument represents the number of packes that are going to send.

In [ ]:
mriPacket_send = h1.execute_thread(f"sudo python3 sendPacket.py 1 2 30")

The sleep(10) function call in your code causes the program to pause or delay its execution for 10 seconds. 

In [ ]:
sleep(10)

The Python code snippet runs a script named "BgT_iperf.py" on the host "h1", passing three arguments: h2_ip, tiperf, and nthreads. "h2_ip" represents the ip of the destination host. "tiperf" id the duration of the test in seconds, specified by the tiperf variable. Finally "nthreads" is the number of parallel client threads to use during the iperf test, specified by the nthreads variable.

In [ ]:
iperf_h1 = h1.execute(f"python3 BgT_iperf.py {h2_ip} {tiperf} {nthreads}")

The sleep(10) function call in your code causes the program to pause or delay its execution for 10 seconds. 

In [ ]:
sleep(10)

The code snippet processes the JSON output from an iperf network test, formats it for readability, and saves it to a file. It begins by parsing the JSON output stored in iperf_h1[0] into a Python dictionary. Then, it converts this dictionary back into a JSON-formatted string with indentation for clarity. Finally, it writes this formatted JSON data to a file named according to the experiment number, ensuring the results are neatly saved and organized.

In [ ]:
data_dict = json.loads(iperf_h1[0])
json_data = json.dumps(data_dict, indent=4)
with open(f'./graphs/iperf/exp{experimentNumber}.json', 'w') as file:
    file.write(json_data)

The code snippet processes the output of the receive.py program, formats it for readability, and saves it to a file. 

In [ ]:
stdout, stderr = h2.execute("sudo killall python3", quiet=True)
hex_list = mriPacket_receive.result()[0].split('\n')[1:-1]
df = pd.DataFrame(hex_list, columns=['Values'])
df.to_csv(f'./graphs/iperf/exp{experimentNumber}.csv', index=False)

The following lines executes a command to terminate all tcpdump processes silently in all the nodes of the experiment. 

In [ ]:
stdout, stderr = h1.execute("sudo killall tcpdump", quiet=True)
stdout, stderr = h2.execute("sudo killall tcpdump", quiet=True)
stdout, stderr = s1.execute("sudo killall tcpdump", quiet=True)
stdout, stderr = s2.execute("sudo killall tcpdump", quiet=True)

The following code process the pcap output of each of the tcpdump commands and extract the values of the number of packets of each type and the size of bytes.

In [ ]:
h1_0 = h1.execute("sudo python3 nPackets.py h1_0.pcap")
h2_0 = h2.execute("sudo python3 nPackets.py h2_0.pcap")
print("hosts")
s1_1 = s1.execute("sudo python3 nPackets.py s1_1.pcap")
s1_2 = s1.execute("sudo python3 nPackets.py s1_2.pcap")
print("s1")
s2_1 = s2.execute("sudo python3 nPackets.py s2_1.pcap")
s2_2 = s2.execute("sudo python3 nPackets.py s2_2.pcap")
print("s2")

In the following code we are going to take the outputs of the cell above and store them in a pandas dataframe. 

In [ ]:
array_string = h1_0[0].strip()
data = json.loads(array_string)
dataframeh1_0 = pd.DataFrame(data).T
dataframeh1_0.columns = ['h1_MRI', 'h1_iperf']

array_string = h2_0[0].strip()
data = json.loads(array_string)
dataframeh2_0 = pd.DataFrame(data).T
dataframeh2_0.columns = ['h2_MRI', 'h2_iperf']

dataframe_hosts = pd.concat([dataframeh1_0, dataframeh2_0], axis=1)

array_string = s1_1[0].strip()
data = json.loads(array_string)
dataframes1_1 = pd.DataFrame(data).T
dataframes1_1.columns = ['s11_MRI', 's11_iperf']

array_string = s1_2[0].strip()
data = json.loads(array_string)
dataframes1_2 = pd.DataFrame(data).T
dataframes1_2.columns = ['s12_MRI', 's12_iperf']

dataframe_s1 = pd.concat([dataframes1_1, dataframes1_2], axis=1)

array_string = s2_1[0].strip()
data = json.loads(array_string)
dataframes2_1 = pd.DataFrame(data).T
dataframes2_1.columns = ['s21_MRI', 's21_iperf']

array_string = s2_2[0].strip()
data = json.loads(array_string)
dataframes2_2 = pd.DataFrame(data).T
dataframes2_2.columns = ['s22_MRI', 's22_iperf']

dataframe_s2 = pd.concat([dataframes2_1, dataframes2_2], axis=1)

The code snippet saves data from three different DataFrames to CSV files, organizing the results of an experiment based on the experimentNumber variable.

In [ ]:
dataframe_hosts.to_csv(f'./graphs/iperf/exp{experimentNumber}_nH.csv', index=False)
dataframe_s1.to_csv(f'./graphs/iperf/exp{experimentNumber}_nS1.csv', index=False)
dataframe_s2.to_csv(f'./graphs/iperf/exp{experimentNumber}_nS2.csv', index=False)